In [1]:
import osmium
import shapely.wkb as wkblib
#import osmnx as ox
import geopandas
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import random

In [52]:
#load any country you need -- exchange "Sweden" for some other country at your own risk!
!wget https://download.geofabrik.de/europe/austria-latest.osm.pbf 

--2023-04-17 21:04:13--  https://download.geofabrik.de/europe/austria-latest.osm.pbf
Resolving download.geofabrik.de (download.geofabrik.de)... 65.109.48.72, 65.109.50.43
Connecting to download.geofabrik.de (download.geofabrik.de)|65.109.48.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 715106890 (682M) [application/octet-stream]
Saving to: ‘austria-latest.osm.pbf’

austria-latest.osm. 100%[===================>] 681.98M  61.7MB/s    in 15s     

2023-04-17 21:04:28 (44.8 MB/s) - ‘austria-latest.osm.pbf’ saved [715106890/715106890]



In [53]:
#you will need docker for this cell, but you can also skip this, it makes loading the osm-file faster
!docker run -d -w /wkd -v $(pwd):/wkd stefda/osmium-tool osmium tags-filter -o austria-filtered.osm.pbf austria-latest.osm.pbf building

c2708f48418fa75ea1dae9256db34ad605980d5c07c894a2cbc41dca25d44ffa


In [2]:
class BuildingHandler(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.nodes_count = 0
        self.nodes = []
        self.building_count = 0
        self.buildings = []
        # A global factory that creates WKB from a osmium geometry
        self.wkbfab = osmium.geom.WKBFactory()

    def node(self, n):
        if n.tags.get("building") == 'yes':
            try:
                wkb = self.wkbfab.create_point(n)
                geo = wkblib.loads(wkb, hex=True)
            except Exception as e:
                print(e)
                return
            row = { "w_id": n.id, "geometry": geo }
            
            for key, value in n.tags:
                row[key] = value
                
            self.nodes.append(row)
            self.nodes_count += 1
        
    def area(self, w):
        if w.tags.get("building") == 'yes':
            try:
                wkb = self.wkbfab.create_multipolygon(w)
                geo = wkblib.loads(wkb, hex=True)
            except Exception as e:
                print(e)
                return
            row = { "w_id": w.id, "geometry": geo }

            for key, value in w.tags:
                row[key] = value

            self.buildings.append(row)
            self.building_count += 1

In [3]:
file = "sweden-filtered.osm.pbf"

In [3]:
file2 = "austria-filtered.osm.pbf"

In [4]:
def build_geodf(file):
    
    buildinghandler = BuildingHandler()
    buildinghandler.apply_file(file, locations=True)
    
    i = 200000
    while i-200000 < len(buildinghandler.buildings):
        dfx = pd.DataFrame(buildinghandler.buildings[(i-200000):min([i, len(buildinghandler.buildings)-1])])
        gdfx = geopandas.GeoDataFrame(dfx, geometry='geometry')
        gdfx = gdfx.set_crs("EPSG:4326")
        #gdfx = ox.project_gdf(gdfx)
        #gdfx = gdfx.dropna(subset=['building:levels'])
        gdfx = gdfx[['w_id', 'geometry', 'building:levels']]
        if i < 200001:
            meta = gdfx
        else:
            meta = pd.concat([meta, gdfx])
        print(meta.shape)
        i += 200000
    
    return meta

In [5]:
df = build_geodf(file)

invalid area (area_id=2323208800)
invalid area (area_id=2324227006)
(200000, 3)
(400000, 3)
(600000, 3)
(800000, 3)
(1000000, 3)
(1200000, 3)
(1400000, 3)
(1600000, 3)
(1800000, 3)
(2000000, 3)
(2103704, 3)


In [ ]:
df2 = build_geodf(file2)

(200000, 3)


In [7]:
geodf = df.copy()

In [8]:
geodf["geometry"] = geodf["geometry"].to_crs("EPSG:3857") # to meters instead of lat, lon degees
geodf.loc[geodf['building:levels'].str.contains('[A-Za-z]', na=False)] = None
geodf.loc[geodf['building:levels'].str.contains('[;,.-]', na=False)] = None
geodf.loc[geodf['building:levels'] == "0"] = None
geodf["building:levels"] = geodf["building:levels"].astype("float")

In [9]:
areas = geodf["geometry"].area
locations = geodf["geometry"].centroid
levels = geodf["building:levels"]
data = np.array([areas, locations.x, locations.y, levels]).T
data

array([[1.38563299e+05, 1.36866770e+06, 7.89792886e+06,            nan],
       [6.52691768e+03, 1.37483306e+06, 8.03408778e+06,            nan],
       [4.80110734e+04, 1.99577001e+06, 8.32098602e+06, 4.00000000e+00],
       ...,
       [3.47616274e+02, 1.45504725e+06, 7.66865030e+06,            nan],
       [5.20651090e+02, 1.45507190e+06, 7.66869958e+06,            nan],
       [2.24230732e+02, 1.45505722e+06, 7.66871870e+06,            nan]])

In [10]:
data = data[np.where(~np.isnan(data[:,3]))]

In [11]:
x = data[:, :3]
y = data[:, 3]

In [12]:
from sklearn.model_selection import train_test_split, cross_val_score

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, train_size=10000, random_state=42, shuffle=True
)

In [14]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(criterion='absolute_error')
pred = rfr.fit(X_train, y_train).predict(X_test)
pred

array([1.08, 1.85, 1.99, ..., 3.21, 1.76, 4.54])

In [19]:
np.abs((np.rint(pred)- y_test)).mean()

0.5665789473684211